In [ ]:
// GPU Training Test for Housing Dataset

import hs.ml.model.Model
import hs.ml.model.nn.Dense
import hs.ml.model.nn.activation.ReLU
import hs.ml.autograd.Node
import hs.ml.importer.CsvImporter
import hs.ml.preprocessing.DataPreprocessor
import hs.ml.preprocessing.policy.ReplaceToAvgPolicy
import hs.ml.scaler.MinMaxScaler
import hs.ml.data.DataPipeline
import hs.ml.train.Trainer
import hs.ml.loss.MeanSquaredError
import hs.ml.metric.RootMeanSquaredError
import hs.ml.train.optimizer.Adam
import hs.ml.math.TensorFactory
import hs.ml.math.metal.MetalConfig

// Enable GPU acceleration
TensorFactory.useGpu = true
MetalConfig.enabled = true
MetalConfig.minSizeForGPU = 100

println("=== GPU Training Test ===")
println("GPU Enabled: ${TensorFactory.useGpu}")
println("Metal Available: ${MetalConfig.isAvailable()}")
println("Metal Backend: ${MetalConfig.getBackend()}")
println()

// Define the neural network model
class HousingModel(inputSize: Int) : Model() {
    private val fc1 = Dense(inputSize, 32)
    private val fc2 = Dense(32, 16)
    private val output = Dense(16, 1)
    private val relu = ReLU()

    override fun forward(x: Node): Node {
        var h = fc1.forward(x)
        h = relu.forward(h)
        h = fc2.forward(h)
        h = relu.forward(h)
        return output.forward(h)
    }

    override fun params(): List<Node> {
        return fc1.params() + fc2.params() + output.params()
    }
}

// Load and preprocess data
println("Loading housing dataset...")
val importer = CsvImporter("data/housing.csv")
val preprocessor = DataPreprocessor(
    missingPolicy = ReplaceToAvgPolicy(),
    scaler = MinMaxScaler()
)
val pipeline = DataPipeline(importer, preprocessor)
val data = pipeline.run()

println("Data shape: ${data.inputs.row} x ${data.inputs.col}")
println("Labels shape: ${data.labels.row} x ${data.labels.col}")
println()

// Split data into train and test sets (80/20 split)
val splitIndex = (data.inputs.row * 0.8).toInt()
val trainInputs = TensorFactory.create(splitIndex, data.inputs.col) { i, j -> data.inputs[i, j] }
val trainLabels = TensorFactory.create(splitIndex, 1) { i, _ -> data.labels[i, 0] }
val testInputs = TensorFactory.create(data.inputs.row - splitIndex, data.inputs.col) { i, j -> 
    data.inputs[i + splitIndex, j] 
}
val testLabels = TensorFactory.create(data.inputs.row - splitIndex, 1) { i, _ -> 
    data.labels[i + splitIndex, 0] 
}

val trainBatch = hs.ml.data.DataBatch(trainInputs, trainLabels)
val testBatch = hs.ml.data.DataBatch(testInputs, testLabels)

println("Train set: ${trainBatch.inputs.row} samples")
println("Test set: ${testBatch.inputs.row} samples")
println()

// Create and configure model
val model = HousingModel(data.inputs.col)
model.param.loss = MeanSquaredError()
model.param.optimizer = Adam(lr = 0.001)
model.param.metric = mutableListOf(RootMeanSquaredError())

// Create trainer
val trainer = Trainer(model)

// Training callback to print progress
val verbose: (Int, String) -> Unit = { epoch, msg ->
    println("Epoch $epoch: $msg")
}

println("Starting GPU training...")
val startTime = System.currentTimeMillis()

trainer.train(
    train = trainBatch,
    test = testBatch,
    epochs = 1000,
    verbose = verbose,
    evalEpoch = 100
)

val endTime = System.currentTimeMillis()
val trainingTime = (endTime - startTime) / 1000.0

println()
println("=== Training Complete ===")
println("Total training time: ${trainingTime}s")
println("Final epoch: ${model.epoch}")
println("Model trained: ${model.isTrained}")
println()

// Final evaluation
println("=== Final Evaluation ===")
val finalMetrics = trainer.evaluate(testBatch)
finalMetrics.forEach { (name, value) ->
    println("$name: ${String.format("%.6f", value)}")
}

=== GPU Training Test ===
GPU Enabled: true
Metal Available: false
Metal Backend: null

Loading housing dataset...
DataPreprocessor: 전처리 시작...
DataPreprocessor: 전처리 완료.
Data shape: 20640 x 8
Labels shape: 20640 x 1

Train set: 16512 samples
Test set: 4128 samples

Starting GPU training...
Epoch 1: Training started: Epoch 1 to 1000
